# Lab 7: Extracting traffic information from surveillance cameras

In [ ]:
import os
import supervision
from supervision.video.source import get_video_frames_generator
import cv2 as cv
import numpy as np
import pickle
from scipy import interpolate
import numpy as np
import pandas as pd
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from scipy import cluster
from sklearn.preprocessing import LabelEncoder
import random
import csv

In [ ]:
SOURCE_VIDEO_PATH = "video-traffic.mp4"
DETECTIONS_PATH = 'detections.csv'

### Reading and displaying a video

In [ ]:
path_video1 = os.path.join('',SOURCE_VIDEO_PATH)
cap = cv.VideoCapture(path_video1) 

if cap.isOpened() == False: 
    print("Error opening video stream or file") 
    
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
print('frame_width = ' + str(frame_width))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
print('frame_height = ' + str(frame_height))

length = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
print("total number of frames = " + str(length))

fps = int(cap.get(cv.CAP_PROP_FPS))
print("frames per second = " + str(fps))

cap.release()

In [ ]:
def display_video(video_path: str, max_number_of_frame_to_run: int = None) -> None:
    current_frame = 0 
    
    cap = cv.VideoCapture(path_video1)

    while cap.isOpened(): 

        ret, frame = cap.read()
        if ret is True:
            current_frame = current_frame + 1 
            cv.imshow("Frame", frame)

            if max_number_of_frame_to_run is not None and current_frame > max_number_of_frame_to_run:
                break

            if cv.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv.destroyAllWindows()

In [ ]:
display_video(video_path=SOURCE_VIDEO_PATH, max_number_of_frame_to_run=750)

### Writing a video

In [ ]:
def read_frames(video_path):
    """
    This function takes the video path and returns the a list of frames.
    :param video_path: Path to the video
    """
    frames = []
    cap = cv.VideoCapture(video_path)  
    if cap.isOpened() == False: 
        raise Exception("Error opening video stream or file") 
        return frames
    
    while cap.isOpened():  
        ret, frame = cap.read() # Read the frame
        if ret is True:
            frames.append(frame)
        else:
            break
    cap.release()
    return frames

### Writing frames in folder

In [ ]:
def write_frames_in_folder(video_path):
    
    def ensure_dir(directory):
        if not os.path.exists(directory):
            os.makedirs(directory)

    video_file = SOURCE_VIDEO_PATH
    cap = cv.VideoCapture(video_file)


    output_dir = 'output_frames/'
    ensure_dir(output_dir)

    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break


        frame_filename = os.path.join(output_dir, f'frame_'+str(frame_index)+'.png')
        cv.imwrite(frame_filename, frame)

        frame_index += 1

    cap.release()
    cv.destroyAllWindows()

In [ ]:
write_frames_in_folder(SOURCE_VIDEO_PATH)

### Plot detections in video

In [ ]:
def plot_bbox_in_video(video_path, csv_path):
    def get_random_color():
        return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    data = pd.read_csv(csv_path)
    cap = cv.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_delay = int(1000 / fps)
    
    track_colors = {}
    frame_index = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_data = data[data['frame'] == frame_index]
       
        for index, row in frame_data.iterrows():
            track_id = row['track_id']

            if track_id not in track_colors:
                track_colors[track_id] = get_random_color()

            color = track_colors[track_id]
            x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])
            cv.rectangle(frame, (x1, y1), (x2, y2), color, 2)

        cv.imshow('Video with Bounding Boxes', frame)

        if cv.waitKey(frame_delay) & 0xFF == ord('q'):
            break

        frame_index += 1
        
        if frame_index % 100 == 0:
            print(f"Processed {frame_index} frames")
        
       
        if frame_index % 500 == 0:
            cv.waitKey(500)  

    cap.release()
    cv.destroyAllWindows()



In [ ]:
plot_bbox_in_video(SOURCE_VIDEO_PATH,DETECTIONS_PATH)

In [ ]:
def plot_center_bbox_in_video(video_path, csv_path):
    def get_random_color():
        return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    data = pd.read_csv(csv_path)
    cap = cv.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    fps = cap.get(cv.CAP_PROP_FPS)
    frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    frame_delay = int(1000 / fps)  # Delay in milliseconds

    track_colors = {}
    track_centers = {}

    frame_index = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        white_frame = 255 * np.ones(shape=[frame_height, frame_width, 3], dtype=np.uint8)

        frame_data = data[data['frame'] == frame_index]

        for index, row in frame_data.iterrows():
            track_id = row['track_id']
            
            if track_id not in track_colors:
                track_colors[track_id] = get_random_color()

            color = track_colors[track_id]
            x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])

            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            if track_id not in track_centers:
                track_centers[track_id] = []
            track_centers[track_id].append((center_x, center_y))

        for track_id, centers in track_centers.items():
            color = track_colors[track_id]
            for center in centers:
                cv.circle(white_frame, center, radius=5, color=color, thickness=-1)

        cv.imshow('Trajectories on White Background', white_frame)

        if cv.waitKey(frame_delay) & 0xFF == ord('q'):
            break

        frame_index += 1

    cap.release()
    cv.destroyAllWindows()



In [ ]:
plot_center_bbox_in_video(SOURCE_VIDEO_PATH,DETECTIONS_PATH)

### Calculate and filter the trajectories

In [ ]:
def calculate_trajectories(csv_path):
    centers_dict = {}
    with open(csv_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            track_id = int(row["track_id"])
            cx = (float(row["x1"]) + float(row["x2"])) / 2
            cy = (float(row["y1"]) + float(row["y2"])) / 2

            if track_id not in centers_dict:
                centers_dict[track_id] = []

            centers_dict[track_id].append([cx, cy])

    trajectories=[]
    #TODO:
            
    return trajectories

In [ ]:
trajectories=calculate_trajectories(DETECTIONS_PATH)

In [ ]:
print(f"Number of trajectories: {len(trajectories)}")

lengths = []
for trajectory in trajectories:
    lengths.append(len(trajectory))
median_length = np.median(lengths)
print(f"Median number of segments in a trajectory: {median_length}")


segment_length = []
for trajectory in trajectories:
    s = 0
    for i in range(1, len(trajectory)):
        s += np.linalg.norm(trajectory[i] - trajectory[i-1])
    segment_length.append(s)

median_segment_length = np.median(segment_length)
print(f"Median length of the trajectories: {median_segment_length}")

### Reduce trajectory to 150 points

In [ ]:
def reduce_trajectory(trajectory, num_points):
    if not isinstance(trajectory, np.ndarray):
        raise TypeError("Trajectory must be a numpy array")
    elif trajectory.shape[1] != 2:
        raise ValueError("Trajectory must be a numpy array of shape (n, 2)")
    
    if trajectory.shape[0] == num_points:
        return trajectory
    old_points = np.linspace(0, 1, len(trajectory))
    
    new_points = np.linspace(0, 1, num_points)

    new_trajectory = []
    for dimension in range(len(trajectory[0])):
        old_values = [point[dimension] for point in trajectory]
        interpolator = interpolate.PPoly.from_spline(interpolate.splrep(old_points, old_values))
        new_values = interpolator(new_points)
        new_trajectory.append(new_values)

    return np.transpose(new_trajectory)


In [ ]:
def filter_length(trajectory, min_length=50):
    length = 0
    for i in range(1, len(trajectory)):
        length += dist.euclidean(trajectory[i], trajectory[i-1])

    return length >= min_length

In [ ]:
simplified_trajectories = [trajectory for trajectory in simplified_trajectories if filter_length(trajectory, min_length=200)]
first_points = [trajectory[:10] for trajectory in simplified_trajectories]
last_points = [trajectory[-10:] for trajectory in simplified_trajectories]

In [ ]:
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)

In [ ]:
for i, trajectory in enumerate(first_points):
    plt.plot(trajectory[:, 0], trajectory[:, 1])

frame = next(iter(generator))
plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
plt.show()


In [ ]:
#TODO Plot the last points

### Apply TSNE 

In [ ]:
np_trajectories = np.array(first_points)

trajectories_flattened = np.array([traj.flatten() for traj in np_trajectories])
n_samples = len(trajectories_flattened)
print(f"Number of samples: {n_samples}")
perplexity = n_samples //5
tsne = TSNE(n_components=2, random_state=0, perplexity=perplexity, n_iter=1000, learning_rate=300)

In [ ]:
trajectories_tsne_first = tsne.fit_transform(trajectories_flattened)
plt.scatter(trajectories_tsne_first[:, 0], trajectories_tsne_first[:, 1])
plt.show()

In [ ]:
np_trajectories = np.array(last_points)
trajectories_flattened = np.array([traj.flatten() for traj in np_trajectories])

trajectories_tsne_last = tsne.fit_transform(trajectories_flattened)

plt.scatter(trajectories_tsne_last[:, 0], trajectories_tsne_last[:, 1])
plt.show()


In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
trajectories_tsne_first_standardized = scaler.fit_transform(trajectories_tsne_first)
trajectories_tsne_last_standardized = scaler.transform(trajectories_tsne_last)

trajectories_tsne_first_standardized = (trajectories_tsne_first_standardized - np.min(trajectories_tsne_first_standardized)) / (np.max(trajectories_tsne_first_standardized) - np.min(trajectories_tsne_first_standardized))
trajectories_tsne_last_standardized = (trajectories_tsne_last_standardized - np.min(trajectories_tsne_last_standardized)) / (np.max(trajectories_tsne_last_standardized) - np.min(trajectories_tsne_last_standardized))

plt.scatter(trajectories_tsne_first_standardized[:, 0], trajectories_tsne_first_standardized[:, 1])
plt.show()

plt.scatter(trajectories_tsne_last_standardized[:, 0], trajectories_tsne_last_standardized[:, 1])
plt.show()

In [ ]:
cluster_model =KMeans(n_clusters=4)

### Clusterize the first points

In [ ]:
cluster_assignments = cluster_model.fit_predict(trajectories_tsne_first_standardized)
plt.scatter(trajectories_tsne_first_standardized[:, 0], trajectories_tsne_first_standardized[:, 1], c=cluster_assignments)


for i in np.unique(cluster_assignments):
    cluster_points = trajectories_tsne_first_standardized[cluster_assignments == i]
    x = np.mean(cluster_points[:, 0])
    y = np.mean(cluster_points[:, 1])
    plt.text(x, y, str(i), fontsize=12, color='red')
    plt.gca().add_artist(plt.Circle((x, y), 0.1, color='red', fill=False))

plt.show()

### Clusterize the last points

In [ ]:
#TODO Clusterize the last points

In [ ]:
for cluster_id in np.unique(cluster_assignments):
    cluster_indices = np.where(cluster_assignments == cluster_id)[0]
    for i in cluster_indices:
        traj = first_points[i]
        plt.plot(traj[:, 0], traj[:, 1], color = plt.cm.tab20(cluster_id))
generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
frame = next(iter(generator))


for cluster_id in np.unique(cluster_assignments):
    cluster_indices = np.where(cluster_assignments == cluster_id)[0]
    plt.figure(figsize=(10, 6))
    
    for i in cluster_indices:
        
        traj = first_points[i]
        
        plt.plot(traj[:, 0], traj[:, 1], color = plt.cm.tab20(cluster_id))

    plt.title(f"Cluster {cluster_id}")
    plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    plt.show()

plt.figure(figsize=(10, 6))


In [ ]:
for cluster_id in np.unique(cluster_assignments_last):
    cluster_indices = np.where(cluster_assignments_last == cluster_id)[0]
    plt.figure(figsize=(10, 6))

    for i in cluster_indices:
        traj = last_points[i]

        plt.plot(traj[:, 0], traj[:, 1], color = plt.cm.tab20(cluster_id))

    plt.title(f"Cluster {cluster_id}")
    plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    plt.show()

plt.figure(figsize=(10, 6))



In [ ]:
for cluster_id in np.unique(cluster_assignments):
    cluster_indices = np.where(cluster_assignments == cluster_id)[0]

    for i in cluster_indices:
        traj = first_points[i]
        plt.plot(traj[:, 0], traj[:, 1], color = plt.cm.tab20(cluster_id))

for cluster_id in np.unique(cluster_assignments_last):
    cluster_indices = np.where(cluster_assignments_last == cluster_id)[0]
    
    for i in cluster_indices:
        traj = last_points[i]
        plt.plot(traj[:, 0], traj[:, 1], color = plt.cm.tab20(cluster_id + 4))

In [ ]:
first_clusters = cluster_assignments
last_clusters = cluster_assignments_last

all_trajectory_clusters = {}


#TODO plot directions 


for cluster in all_trajectory_clusters.keys():
    print(f"Cluster {cluster}: {len(all_trajectory_clusters[cluster])} trajectories")
    plt.figure(figsize=(10, 6))
    for trajectory in all_trajectory_clusters[cluster]:
        plt.plot(trajectory[:, 0], trajectory[:, 1])

    plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    plt.show()



### Average the trajectory

In [ ]:
for cluster in all_trajectory_clusters:
    trajectories = all_trajectory_clusters[cluster]

In [ ]:
def average_trajectory(trajectories):
    return np.mean(trajectories, axis=0)

average_trajectories = {}
for cluster_key, trajectories in all_trajectory_clusters.items():
    trajectory_stack = np.stack(trajectories)
    average_trajectories[cluster_key] = average_trajectory(trajectory_stack)

plt.figure(figsize=(10, 6))
for cluster_key, avg_traj in average_trajectories.items():
    plt.plot(avg_traj[:, 0], avg_traj[:, 1], label=f"Cluster {cluster_key}")
plt.legend()
plt.title("Average Trajectories for Each Cluster")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
for cluster_key, avg_traj in average_trajectories.items():
    plt.plot(avg_traj[:, 0], avg_traj[:, 1], label=f"Cluster {cluster_key}")
plt.legend()
plt.title("Average Trajectories for Each Cluster")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
plt.show()